In [ ]:
import requests
import json
import time

Authenticate with Twitter and get a bearer token:

In [ ]:
with open("twitter.secret.json",encoding = 'utf-8') as f:
    credentials = json.loads(f.read())

r = requests.post('https://api.twitter.com/oauth2/token?grant_type=client_credentials', auth=(credentials['api_key'],credentials['api_secret']))

access_token = r.json()["access_token"]

A function for downloading a person's friends (the people they follow).  The API is limited to 15 requests/15 minutes, so retry logic is included. 

In [ ]:
headers = {'Authorization':'Bearer {token}'.format(token=access_token)}

def download_friends(screen_name,next_cursor):
    friends = []
    max_retry_count = 0 # create a max retry count so we don't have an infinite loop

    while(next_cursor != 0 ):
        r = requests.get('https://api.twitter.com/1.1/friends/list.json?cursor={cursor}&screen_name={screen_name}&include_user_entities=true&skip_status=true&count=200'.format(cursor=next_cursor, screen_name=screen_name), headers=headers)
    
        if (r.status_code == 200):
            data = r.json()
            data_friends = [{ "id_str": user["id_str"], "screen_name":user["screen_name"], "friends_count":user["friends_count"]} for user in data["users"]]
            friends.extend(data_friends)

            next_cursor = data["next_cursor"]
            max_retry_count = 0
        elif (r.status_code == 401):
            next_cursor = 0
        else:
            max_retry_count = max_retry_count + 1
            print(r.status_code)
            print(max_retry_count)
            if (max_retry_count >= 20):
                raise Exception('max_retry_count limit reached')
            time.sleep(60)
    
    return friends

Download the friends for our initial user:

In [ ]:

screen_name="bertwagner"
friends = download_friends(screen_name,-1)

with open('./output_friends/{screen_name}.json'.format(screen_name=screen_name), 'w') as file:
    file.write(json.dumps(friends))

Download the friends of the initial user's friends.  We skip friends that have more than 3000 friends because those are probably not going to be accounts we are interested in for this analysis (someone who follows that many people proabbly isn't actually reading their Twitter feed).  This process is slow and took about 24 hours for the 450 people I follow (the Twitter API rate limits really slow things down):

In [ ]:
for friend in friends:
    screen_name = friend["screen_name"]

    friends_count = friend["friends_count"]
    if (friends_count <= 3000):
        indirect_friends = download_friends(screen_name,-1)

        with open('./output_friends/{screen_name}.json'.format(screen_name=screen_name), 'w') as file:
            file.write(json.dumps(indirect_friends))